In [6]:
import sys
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import lxml.html as lh

# Scaping Data from Wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
response = requests.get(url)

In [4]:
response.raise_for_status()

### Parsing the scraped Data between <tr>  and </tr> of html

In [17]:
doc = lh.fromstring(response.content)

tr_values = doc.xpath('//tr')

### Creating Header and empty List

In [18]:
tr_values= doc.xpath('//tr')
col=[]
i=0

for t in tr_values[0]:
    i+=1
    head =t.text_content()
    print ('%d:"%s"'%(i,head))
    col.append((head,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


### Storing data from second row

In [19]:
for k in range(1,len(tr_values)):
    
    Hd=tr_values[k]
     #If row is not of size 3, the //tr data is not from our table 
    if len(Hd)!=3:
        break
    i=0  #index of our column
    
    
    for t in Hd.iterchildren(): #Iterate through each element of the row
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [20]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [11]:
## Create the data frame

In [33]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [34]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [35]:
df = df.replace('\n','', regex=True)
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [36]:
df.columns.values

array(['Postal Code\n', 'Borough\n', 'Neighborhood\n'], dtype=object)

In [37]:
cols= ['Post Code', 'Borough','Neighborhood']

In [38]:
df.columns =cols
df.head()

,Post Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [39]:
df = df[df.Borough!= 'Not assigned']
df.head(10)

,Post Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [40]:
df.shape

(104, 3)